## LSTM from scratch

In [0]:
#!pip install opencv-python
#!apt update && apt install -y libsm6 libxext6
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
! pip3 install fastai
#!pip3 install torchvision
! pip install torchtext

In [ ]:
! pip install spacy

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *
from fastai.column_data import *

PATH='data/nietzsche/'
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

nietzsche.txt: 606kB [00:00, 1.01MB/s]                           

corpus length: 600893


In [0]:
TRN='data/nietzsche/trn'
VAL='data/nietzsche/val'

os.makedirs(TRN, exist_ok=True)
os.makedirs(VAL, exist_ok=True)

train_perc = .8
with open(f'{PATH}/nietzsche.txt', 'r') as fp:
    lines = fp.readlines()
    text_len = len(lines)
    part_train = open(f'{TRN}nietzsche1.txt', 'w')
    part_val = open(f'{VAL}nietzsche2.txt', 'w')    
    for ix,l in enumerate(lines):

        if ix/text_len<train_perc:
            part_train.write(l)
        else:
            part_val.write(l)
    part_train.close()
    part_val.close()    

In [0]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH='data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

In [0]:
! mv data/nietzsche/trnnietzsche1.txt data/nietzsche/trn
! mv data/nietzsche/valnietzsche2.txt data/nietzsche/val

In [59]:
ls data/nietzsche


models/  nietzsche.txt  trn/  val/


In [0]:
TEXT= data.Field(lower=True,tokenize=list)
bs=64; bptt= 8; n_fac=42; n_hidden=512
                 

In [0]:
FILES= dict(train=TRN_PATH,validation=VAL_PATH,test=VAL_PATH)
md=LanguageModelData.from_text_files(PATH,TEXT,**FILES,bs=bs,bptt=bptt,min_freq=3)

## LSTM-Long Short Term Memory
**Using LSTM we can generate multiple characters of output as LSTM will also save the state. For this we use non overlapping data unlike last what we did for the last char predictions model**

In [0]:
from fastai import sgdr


In [0]:
# Functions to test our model
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = model(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [0]:
class charlstm(nn.Module):
  def __init__(self,vocab_size,n_fac,bs,nl):
    super().__init__()
    self.vocab_size=vocab_size
    self.nl=nl
    self.e=nn.Embedding(vocab_size,n_fac)
    self.rnn=nn.LSTM(n_fac,n_hidden,nl,dropout=0.5)
    self.l_out=nn.Linear(n_hidden,vocab_size)
    self.init_hidden(bs)

  def init_hidden(self,bs):
    self.h=(V(torch.zeros(self.nl, bs, n_hidden)),V(torch.zeros(self.nl, bs, n_hidden)))
    
  def forward(self,cs):
    bs=cs[0].size(0)
    if self.h[0].size(1)!= bs: self.init_hidden(bs)
    outp,h=self.rnn(self.e(cs),self.h)
    self.h=repackage_var(h)
    res=F.log_softmax(self.l_out(outp),dim= -1).view(-1,self.vocab_size)
    return res
    

In [0]:
model=charlstm(md.nt,n_fac,512,2).cuda()
lo= LayerOptimizer(optim.Adam,model,1e-2, 1e-5) #fastai class to set optimizer and use callbacks for cosine Annealing

In [0]:
os.makedirs(f'{PATH}models', exist_ok=True)
on_end = lambda sched, cycle: save_model(model, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)] #callbacks- Cosine Anealing(to decrease the lr while raining)


In [63]:
fit(model,md,15,lo.opt,F.nll_loss,callbacks=cb)

epoch      trn_loss   val_loss   
    0      1.627835   1.553091  
 24%|██▍       | 227/942 [00:05<00:17, 41.61it/s, loss=1.72]    1      1.640158   1.557315  
    2      1.457372   1.426731  
    3      1.652228   1.576038  
    4      1.541071   1.48558   
    5      1.43933    1.400615  
  1%|          | 10/942 [00:00<00:26, 35.60it/s, loss=1.45]    6      1.362172   1.363111  
    7      1.621384   1.548219  
    8      1.568522   1.508488  
  0%|          | 0/942 [00:00<?, ?it/s, loss=1.58]    9      1.527331   1.47767   
    10     1.478657   1.446148  
    11     1.427089   1.40347   
    12     1.370189   1.368686  
    13     1.313825   1.339289  
    14     1.279246   1.325342  



[array([1.32534])]

In [64]:
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)] 
fit(model,md,35,lo.opt,F.nll_loss,callbacks=cb)

epoch      trn_loss   val_loss   
    0      1.262232   1.323915  
 25%|██▍       | 232/942 [00:05<00:16, 43.88it/s, loss=1.24]    1      1.263847   1.322543  
    2      1.26379    1.322015  
    3      1.266319   1.320333  
    4      1.258633   1.318609  
  1%|          | 10/942 [00:00<00:20, 44.79it/s, loss=1.27]    5      1.252031   1.318211  
    6      1.252289   1.317534  
    7      1.251999   1.317608  
    8      1.250888   1.315871  
    9      1.238092   1.315004  
    10     1.236999   1.312757  
    11     1.22699    1.312791  
    12     1.225057   1.312     
    13     1.216242   1.311733  
  1%|          | 5/942 [00:00<00:38, 24.29it/s, loss=1.23]    14     1.225105   1.311203  
    15     1.228411   1.312599  
    16     1.221586   1.312049  
    17     1.220585   1.311043  
    18     1.213573   1.311281  
  0%|          | 0/942 [00:00<?, ?it/s]    19     1.204911   1.310851  
    20     1.204461   1.308008  
    21     1.190805   1.309551  
    22     1.186141   1.

[array([1.31101])]

In [77]:
print(get_next_n('You saved me from', 400))

You saved me from man thanthe foundation: but as any hearts of such a conscience of "the symbolism"; to false however, who is hitherto for a result on thevalue of recognized his christian, charms and stand," to movement: therefore, is then sakewithfrom by the life of authoritative striving, and enjoyments of our organs, whatever sizes to which even they do a matter myself, all the sacrificed "desirable--almostinno


In [78]:
print(get_next_n('I love y', 400))

I love your actually what was have comes to our faith.; finally cost anddevelops of skfrench entennan, ganded to patry, sentiments, and know--the psychological prayers rendered andadvances to a favourableness power, and satisfaction, for one's philosophereze-vanity, will that is well that have insubjecting a god; thee never at thepresent still. saw strong, somethingprobably that the moment ofsympathy and 


## End.